In [25]:
import numpy as np 
from ipywidgets import widgets, interact, interact_manual
from matplotlib import pyplot as plt
import subprocess
import json
import math 
from collections import namedtuple
import time

perf_calc_path = R".\bin\Release\netcoreapp3.1\PerformanceCalculator.dll"



def getData(correction, dstart, dend, dcount, bounds=5, count=50):
    
    dspacing = 0 if dcount==1 else (dend-dstart)/(dcount-1)

    spacing = 2*bounds/(count-1)
    result = subprocess.run(
        f"dotnet {perf_calc_path} visualise {correction} -s {dstart} -c {dcount} -d {dspacing} -s {-bounds} -c {count} -d {spacing} -s {-bounds} -c {count} -d {spacing}",
        encoding="UTF8",
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE    
        )
    try:
        result.check_returncode()
        arr = json.loads(result.stdout)
        return np.array(arr)
    except:
        print(result.stderr)
        print(result.stdout)
        raise

def plot(fig, ax, data, size, title):
    heatmap = ax.imshow(data,origin="lower", extent=(-size,size,-size,size), interpolation="bilinear")
    ax.set_title(title)
    fig.colorbar(heatmap, ax=ax)
    return heatmap

In [28]:
print(subprocess.run(
        f"dotnet build -c Release",
        encoding="UTF8",
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE    
        ).stdout)

'Microsoft (R) Build Engine version 16.5.0+d4cbfca49 for .NET Core\nCopyright (C) Microsoft Corporation. All rights reserved.\n\n  Restore completed in 48.15 ms for c:\\Users\\Joseph\\source\\repos\\osu-tools-delta\\PerformanceCalculator\\PerformanceCalculator.csproj.\n  Restore completed in 47.65 ms for c:\\Users\\Joseph\\source\\repos\\osu-tools-delta\\osu\\osu.Game.Rulesets.Osu\\osu.Game.Rulesets.Osu.csproj.\n  Restore completed in 47.74 ms for c:\\Users\\Joseph\\source\\repos\\osu-tools-delta\\osu\\osu.Game.Rulesets.Mania\\osu.Game.Rulesets.Mania.csproj.\n  Restore completed in 47.63 ms for c:\\Users\\Joseph\\source\\repos\\osu-tools-delta\\osu\\osu.Game.Rulesets.Taiko\\osu.Game.Rulesets.Taiko.csproj.\n  Restore completed in 47.73 ms for c:\\Users\\Joseph\\source\\repos\\osu-tools-delta\\osu\\osu.Game\\osu.Game.csproj.\n  Restore completed in 47.67 ms for c:\\Users\\Joseph\\source\\repos\\osu-tools-delta\\osu\\osu.Game.Rulesets.Catch\\osu.Game.Rulesets.Catch.csproj.\n  osu.Game -> 

In [26]:

@interact_manual(corr=["FLOW_0","SNAP_0","FLOW_3","SNAP_3"], plot_scale=(0,20), d_start=(0.0,20.0), d_end=(0.0,20.0), d_count=(1,10))
def make_plots(corr="FLOW_0", plot_scale=8, d_start=0, d_end=6, d_count=2):
    data = getData(corr, d_start, d_end, d_count, plot_scale)
    old_data = getData(corr+"_OLD",  d_start, d_end, d_count, plot_scale)
    diff_data = data-old_data
    fig, axes = plt.subplots(d_count,3, figsize=(20,4.5*d_count))
    fig.suptitle(f"{corr}")
    step = 0 if d_count==1 else (d_end-d_start)/(d_count-1)
    if d_count==1:
        axes = np.array([axes])
    for i in range(d_count):
        d = d_start+i*step
        new = plot(fig, axes[i, 0], data[i].T, plot_scale, f"Interpolated d={d}")
        old = plot(fig, axes[i, 1], old_data[i].T, plot_scale, f"Old d={d}")
        delta = plot(fig, axes[i, 2], diff_data[i].T, plot_scale, f"Diff d={d}")
    plt.show()





interactive(children=(Dropdown(description='corr', options=('FLOW_0', 'SNAP_0', 'FLOW_3', 'SNAP_3'), value='FL…